Dans ce script, nous allons voir comment utiliser le module spaCy pour détecter les entités nommées dans un document Gallica via l'API de Gallica.
On va utiliser le modèle de langue française de spaCy pour détecter les entités nommées dans le document.
On va aussi utiliser BeautifulSoup pour supprimer les balises HTML du document et récupérer le texte brut pour les traitements NLP.
Nous allons également utiliser le module requests pour récupérer le contenu HTML du document Gallica.
Et après on les nettoie avec BeautifulSoup pour afficher les entités nommées détectées et sqlite3 pour stocker les entités nommées dans une base de données SQLite.


In [ ]:

import spacy
import pandas as pd
from bs4 import BeautifulSoup
import requests
import sqlite3

import requests
from bs4 import BeautifulSoup
import spacy

def traitement_document_gallica(url):
    try:
        # Récupération du contenu HTML du document
        response = requests.get(url)
        text = response.text
        
        # Suppression des balises HTML
        soup = BeautifulSoup(text, "html.parser")
        clean_text = soup.get_text()
        
        # Chargement du modèle de langue française
        nlp = spacy.load("fr_core_news_sm")
        
        # Traitement NER
        doc = nlp(clean_text)
        
        # Affichage des entités nommées détectées
        for ent in doc.ents:
            print(f"{ent.text}: {ent.label_} ({spacy.explain(ent.label_)} ) ")
        
        return doc  # Vous pouvez également retourner le doc ou d'autres données selon vos besoins
        
    except requests.RequestException as e:
        print("Erreur lors de la requête :", str(e))
        return None

# Appel de la méthode pour traiter le document Gallica avec l'URL spécifié
url_gallica = "https://gallica.bnf.fr/ark:/12148/bpt6k1426047z"
traitement_document_gallica(url_gallica)


Ici nous normalisons le texte pour le rendre plus lisible et plus facile à traiter.
On va assi normaliser les entités nommées pour les rendre plus facilement identifiables (par exemple, on va remplacer les dates par la chaîne de caractères "DATE").
Remplacer les espaces multiples par un seul espace


In [2]:
# fonction pour normaliser les entités nommées détectées dans un document Gallica
def normaliser_entites_nommees(url):
    try:
        # Récupération du contenu HTML du document
        response = requests.get(url)
        text = response.text
        
        # Suppression des balises HTML
        soup = BeautifulSoup(text, "html.parser")
        clean_text = soup.get_text()
        
        # Chargement du modèle de langue française
        nlp = spacy.load("fr_core_news_sm")
        
        # Traitement NER
        doc = nlp(clean_text)
        
        # Normalisation des entités nommées détectées
        entites_normalisees = []
        for ent in doc.ents:
            if ent.label_ == "LOC":  # Exemple : normalisation des lieux
                entite_normalisee = normaliser_lieux(ent.text)
                entites_normalisees.append((entite_normalisee, ent.label_))
            elif ent.label_ == "DATE":  # Exemple : normalisation des dates
                entite_normalisee = normaliser_dates(ent.text)
                entites_normalisees.append((entite_normalisee, ent.label_))
            elif ent.label_ == "PER":  # Exemple : normalisation des personnes
                entite_normalisee = normaliser_personnes(ent.text)
                entites_normalisees.append((entite_normalisee, ent.label_))    
            else:
                entites_normalisees.append((ent.text, ent.label_))
        
        return entites_normalisees
        
    except requests.RequestException as e:
        print("Erreur lors de la requête :", str(e))
        return None

# Fonctions de normalisation des lieux
def normaliser_lieux(entite):
    # Par exemple, convertir en majuscules, supprimer les espaces, etc.
    return entite.upper().replace("  ", " ")

# Fonctions de normalisation des dates
def normaliser_dates(entite):
   return entite
def normaliser_personnes(entite):
    # convertir en majuscules, supprimer les espaces en trop, etc.
    return entite.upper().replace("  ", " ")
    
# Appel de la méthode de normalisation des entités nommées pour l'URL spécifiée
url_gallica = "https://gallica.bnf.fr/ark:/12148/bpt6k1426047z"
entites_normalisees = normaliser_entites_nommees(url_gallica)

# Affichage des entités nommées normalisées
if entites_normalisees:
    print("Entités nommées normalisées:")
    for entite, label in entites_normalisees:
        print(f"{entite}: {label}")

Entités nommées normalisées:
ALFRED BONZON: PER
Bourse de Lyon: ORG
GALLICA                                                              : PER
FR   EN   DE   ES: MISC
IT   RU                                        : MISC
BIBLIOTHÈQUE NATIONALE DE FRANCE   : LOC
GALLICA  : PER
GALLICA: MISC
SUGGÉRÉS: MISC
HUGO: PER
VICTOR HUGO: PER
VICTOR-MARIE ATELIERS: PER
ALÉSI HUGO: PER
FRANÇOIS-VICTOR HUGO: PER
ABEL HUGO: PER
CHARLES : PER
HUGO DE SANCTO: PER
HUGO: MISC
VICTORE: MISC
HUGO: PER
JOSEPH: PER
SUGGÉRÉS: MISC
HUGO DE SANCTO: PER
LOUIS : PER
Gentilly  : MISC
HUGO GROTIUS: PER
GEBIETE: PER
Victor: MISC
HUGO DE SANCTO: PER
GÉNÉRAL HUGO: PER
PAPE: PER
VICTOR HUGO: PER
L’EGLISE     RECHERCHE: PER
ACCÉDER: LOC
Recherche avancée     : MISC
Sélections Accéder: ORG
ACCÉDER: LOC
TYPES: LOC
Livres   Manuscrits   Cartes   Images   Presse: MISC
Enregistrements sonores   : MISC
Partitions   : MISC
Vidéos     Thématiques   Arts: ORG
Histoire   Langues   Littératures   Philosophie   : MISC
Panier   Pani

Après avoir normalisé le texte, nous inserons les entités nommées détectées dans une base de données SQLite.
Ou encore les stocker dans un fichier CSV. Cette technique est utile pour l'analyse de corpus et rendra les données reutilisables pour des chercheurs en humanités numériques nou en histoire.

In [3]:
# fonction pour inserer les entites nommées dans une base de données sqlite à parir la liste des entités nommées normalisées
def inserer_entites_nommees(entites_normalisees):
    try:
        # Connexion à la base de données SQLite
        conn = sqlite3.connect("entites.db")
        cursor = conn.cursor()
        
        # Création de la table
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS entites_nommees (
                id INTEGER PRIMARY KEY AUTOINCREMENT UNIQUE,
                entite TEXT,
                label TEXT
            )
        """)
        
        # Insertion des entités nommées dans la base de données
        cursor.executemany("""
            INSERT INTO entites_nommees (entite, label) VALUES (?, ?)
        """, entites_normalisees)
        
        # Sauvegarde de la transaction
        conn.commit()
        
        # Fermeture de la connexion à la base de données
        conn.close()
        
    except sqlite3.Error as e:
        print("Erreur lors de la requête SQL :", str(e))
        return None
# Appel de la méthode pour insérer les entités nommées dans la base de données
inserer_entites_nommees(entites_normalisees) 

Cette partir permet de céer un data frame avec les entités nommées détectées et les afficher dans un tableau.

In [4]:
# stocker les entités nommées normalisées dans un DataFrame
df = pd.DataFrame(entites_normalisees, columns=["entite", "label"])
df.head()
# exporter le DataFrame au format CSV
entite_csv = df.to_csv("entites_normalisees.csv", index=False)

On stocke les entités nommées normalisées dans une base de données SQLite, sans passer par un DataFrame ou une fonction

In [5]:
# stocker les entités nommées normalisées dans une base de données SQLite sans utiliser une fonction
conn = sqlite3.connect("entites_normalisees1.db")
df.to_sql("entites_normalisees", conn, if_exists="replace", index=False)
conn.close()

In [8]:
# Utilisation du modèle pour la classification
"""
Dans cet exemple, la fonction classifier_entites_nommees prend le modèle entraîné et le vectorizer
 (qui ont été obtenus à partir de la préparation des données) ainsi qu'un texte à classifier. 
 Cette fonction vectorise le texte en utilisant le vectorizer, puis utilise le modèle pour prédire 
 la catégorie de l'entité dans le texte donné.
"""
def classifier_entites_nommees(model, vectorizer, texte):
    # Vectorisation du texte avec le vectorizer
    texte_vectorise = vectorizer.transform([texte])
    
    # Prédiction de la catégorie de l'entité
    prediction = model.predict(texte_vectorise)
    
    # Interprétation de la prédiction
    if prediction[0] == 0:
        return "Personne"
    elif prediction[0] == 1:
        return "Organisation"
    else:
        return "Catégorie indéterminée"

# Texte à classifier
texte_a_classifier = "Texte à classer, extrait du document Gallica..."

# Utilisation du modèle pour classifier le texte
categorie = classifier_entites_nommees(model, vectorizer, texte_a_classifier)
print(f"Catégorie de l'entité : {categorie}")


Catégorie de l'entité : Catégorie indéterminée


Ici, on compare les résultats de SpaCy et Flair pour le français sur le même texte
pour les entités de type Personne (PER) et Organisation (ORG).
Cela utilise la bibliothèque Flair pour extraire les entités nommées du texte, puis les compare avec les entités extraites par SpaCy pour permettre une comparaison visuelle. Vous pouvez ajuster les paramètres selon vos besoins spécifiques.

On pour décider par la suite quelle méthode utiliser pour détecter les entités nommées dans les documents Gallica avec moins d'erreurs possibles.

In [6]:
import spacy
from bs4 import BeautifulSoup
import requests
import pandas as pd
from flair.data import Sentence
from flair.models import SequenceTagger

# URL du document sur Gallica
url = "https://gallica.bnf.fr/ark:/12148/bpt6k1426047z"
response = requests.get(url)
text = response.text

# Supprimer les balises HTML
soup = BeautifulSoup(text, "html.parser")
clean_text = soup.get_text()

# Modèle de langue française avec réseaux neuronaux de SpaCy
nlp_spacy = spacy.load("fr_core_news_sm")

# Traitement NER avec SpaCy
doc_spacy = nlp_spacy(clean_text)
entities_spacy = [(ent.text, ent.label_) for ent in doc_spacy.ents if ent.label_ in ['PER', 'ORG']]

# Traitement NER avec Flair
tagger = SequenceTagger.load('ner-ontonotes')
sentence = Sentence(clean_text)
tagger.predict(sentence)
entities_flair = [(entity.text, entity.get_labels()[0].value) for entity in sentence.get_spans('ner')]

# Stockage des entités nommées dans des DataFrames
df_spacy = pd.DataFrame(entities_spacy, columns=['Entité', 'Type (SpaCy)'])
df_flair = pd.DataFrame(entities_flair, columns=['Entité', 'Type (Flair)'])

# Affichage des DataFrames pour comparaison
print("Résultats de SpaCy:")
print(df_spacy.head())  # Affiche les 5 premières entités de SpaCy
print("\nRésultats de Flair:")
print(df_flair.head())  # Affiche les 5 premières entités de Flair


c:\Users\ismal\projets\initiation_a_la_recherche\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



2023-12-26 17:00:25,939 SequenceTagger predicts: Dictionary with 75 tags: O, S-PERSON, B-PERSON, E-PERSON, I-PERSON, S-GPE, B-GPE, E-GPE, I-GPE, S-ORG, B-ORG, E-ORG, I-ORG, S-DATE, B-DATE, E-DATE, I-DATE, S-CARDINAL, B-CARDINAL, E-CARDINAL, I-CARDINAL, S-NORP, B-NORP, E-NORP, I-NORP, S-MONEY, B-MONEY, E-MONEY, I-MONEY, S-PERCENT, B-PERCENT, E-PERCENT, I-PERCENT, S-ORDINAL, B-ORDINAL, E-ORDINAL, I-ORDINAL, S-LOC, B-LOC, E-LOC, I-LOC, S-TIME, B-TIME, E-TIME, I-TIME, S-WORK_OF_ART, B-WORK_OF_ART, E-WORK_OF_ART, I-WORK_OF_ART, S-FAC
Résultats de SpaCy:
                                              Entité Type (SpaCy)
0                                      Alfred Bonzon          PER
1                                     Bourse de Lyon          ORG
2  Gallica                                       ...          PER
3                                         Gallica             PER
4                                               Hugo          PER

Résultats de Flair:
                   Entité T